In [1]:
import pandas as pd
import numpy as np

# Task 0
Data extraction: get the data from 3 tables & combine it into single `.csv` file.
After that read this file using pandas to create Dataframe.
So it will be all joined data in 1 dataframe. Quick check - should be 74818 rows in it.

In [3]:
import sqlite3

conn = sqlite3.connect("db.sqlite3")

query = """
SELECT o.id AS order_id, o.datetime AS order_datetime, 
       oi.quantity, oi.product_id, 
       p.price, p.name AS product_name
FROM restaurant_order AS o
JOIN restaurant_orderitem AS oi ON o.id = oi.order_id
JOIN restaurant_product AS p ON oi.product_id = p.id;
"""
df = pd.read_sql(query, conn)

df.to_csv("restaurant_data.csv", index=False)

conn.close()


DatabaseError: Execution failed on sql '
SELECT o.id AS order_id, o.datetime AS order_datetime, 
       oi.quantity, oi.product_id, 
       p.price, p.name AS product_name
FROM restaurant_order AS o
JOIN restaurant_orderitem AS oi ON o.id = oi.order_id
JOIN restaurant_product AS p ON oi.product_id = p.id;
': no such table: restaurant_order

# Task 1
Get Top 10 most popular products in restaurant sold by Quantity.
Count how many times each product was sold and create a pie chart with percentage of popularity (by quantity) for top 10 of them.

Example:

![pie chart](../demo/pie.png)

In [ ]:
import matplotlib.pyplot as plt

top_products = df.groupby("product_name")["quantity"].sum().nlargest(10)

plt.figure(figsize=(8, 8))
top_products.plot.pie(autopct="%1.1f%%", startangle=90, colors=plt.cm.Paired.colors)
plt.title("Top 10 Products by Quantity")
plt.ylabel("")
plt.show()


# Task 2
Calculate `Item Price` (Product Price * Quantity) for each Order Item in dataframe.
And Make the same Top 10 pie chart, but this time by `Item Price`. So this chart should describe not the most popular products by quantity, but which products (top 10) make the most money for restaurant. It should be also with percentage.

In [ ]:
df["item_price"] = df["price"] * df["quantity"]

top_products_by_price = df.groupby("product_name")["item_price"].sum().nlargest(10)

plt.figure(figsize=(8, 8))
top_products_by_price.plot.pie(autopct="%1.1f%%", startangle=90, colors=plt.cm.Paired.colors)
plt.title("Top 10 Products by Item Price")
plt.ylabel("")
plt.show()


# Task 3
Calculate `Order Hour` based on `Order Datetime`, which will tell about the specific our the order was created (from 0 to 23). Using `Order Hour` create a bar chart, which will tell the total restaurant income based on the hour order was created. So on x-axis - it will be values from 0 to 23 (hours), on y-axis - it will be the total sum of order prices, which were sold on that hour.

Example:

![bar chart](../demo/bar.png)

In [ ]:
df["order_hour"] = pd.to_datetime(df["order_datetime"]).dt.hour

income_by_hour = df.groupby("order_hour")["item_price"].sum()

plt.figure(figsize=(10, 6))
income_by_hour.plot(kind="bar", color="skyblue")
plt.title("Total Income by Order Hour")
plt.xlabel("Hour of Day")
plt.ylabel("Total Income")
plt.xticks(rotation=0)
plt.show()

# Task 4
Make similar bar chart, but right now with `Order Day Of The Week` (from Monday to Sunday), and also analyze total restaurant income by each day of the week.

In [ ]:
df["order_day"] = pd.to_datetime(df["order_datetime"]).dt.day_name()

income_by_day = df.groupby("order_day")["item_price"].sum()

plt.figure(figsize=(10, 6))
income_by_day.plot(kind="bar", color="lightcoral")
plt.title("Total Income by Day of the Week")
plt.xlabel("Day of the Week")
plt.ylabel("Total Income")
plt.xticks(rotation=45)
plt.show()
